<a href="https://colab.research.google.com/github/Msamuelsons/analise-de-dados/blob/main/tratamento_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tópicos para abordar nesse estudo de pré-processamento

In [84]:
import pandas as pd
from io import StringIO
import numpy as np
from sklearn.preprocessing import LabelEncoder

O método *StringIO* Converte uma string em valor(s), em unicode

In [60]:
csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

In [61]:
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [62]:
df.isnull()

,A,B,C,D
0,False,False,False,False
1,False,False,True,False
2,False,False,False,True


Se falta valor o retorno será True, caso contrário False

In [63]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

Uma das maneiras mais fáceis de lidar com dados ausentes é simplesmente remover totalmente os recursos
correspondentes (colunas) ou exemplos de treinamento (linhas) do conjunto de dados; linhas com valores ausentes podem
ser facilmente descartadas através do método dropna

In [64]:
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


axis=0, apaga as linhas com valores ausentes

axis=1, apaga a coluna com valores ausentes

In [65]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


all: Se todos os valores forem NA, elimine essa linha ou coluna.

In [66]:
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


any: Se algum valor de NA estiver presente, elimine essa linha ou coluna.



In [67]:
df.dropna(how='any')

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [68]:
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [69]:
df.dropna(thresh=4) # descarta linhas com menos de 4 valores reais

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [70]:
df.dropna(subset=['C']) #solta apenas as linhas onde NaN aparece em colunas específicas (aqui: 'C')

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


O método de drop(), nem sempre é a melhor opção pois pode acabar apagando muitos dados

# Lidando com valores faltantes

*mean imputation*


---

Uma das técnicas de interpolação mais comuns é a imputação média, onde
simplesmente substituímos o valor ausente pelo valor médio de toda a coluna de recursos.

In [71]:
from sklearn.impute import SimpleImputer
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
dados_amputados = imr.fit_transform(df.values)
dados_amputados

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

Como alternativa, uma maneira ainda mais conveniente de imputar valores ausentes é usar o método fillna dos
pandas e fornecer um método de imputação como argumento. P

In [72]:
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


#Como lidar com dados categóricos?

In [73]:
df_vestuario = pd.DataFrame([
['green', 'M', 10.1, 'class2'],
['red', 'L', 13.5, 'class1'],
['blue', 'XL', 15.3, 'class2']])
df_vestuario.columns = ['color', 'size', 'price', 'classlabel']

In [74]:
size_mapping = {
    'XL': 3,
    'L': 2,
    'M': 1}

In [75]:
df_vestuario['size'] = df_vestuario['size'].map(size_mapping)

In [76]:
df_vestuario

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [77]:
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df_vestuario['size'].map(inv_size_mapping)


0     M
1     L
2    XL
Name: size, dtype: object

Converte a classe em numeral

In [78]:
class_mapping = {
    label: idx for idx, label in enumerate(np.unique(df_vestuario['classlabel']))
}
class_mapping

{'class1': 0, 'class2': 1}

In [79]:
df_vestuario['classlabel'] = df_vestuario['classlabel'].map(class_mapping)
df_vestuario

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


Voltando ao estado original

In [81]:
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df_vestuario['classlabel'] = df_vestuario['classlabel'].map(inv_class_mapping)

In [82]:
df_vestuario

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


Alternativamente, existe uma conveniente classe **LabelEncoder**

In [85]:
class_le = LabelEncoder()
y = class_le.fit_transform(df_vestuario['classlabel'].values)

In [86]:
y

array([1, 0, 1])

Observe que o método fit_transform é apenas um atalho para chamar fit e transform separadamente, e podemos usar o
método inverse_transform para transformar os rótulos das classes inteiras de volta em sua representação de string
original

In [87]:
class_le.inverse_transform(y)

array(['class2', 'class1', 'class2'], dtype=object)